# RAG (Retrieval Augumented Generation) - Example
RAG is a mechanism of retrieving and incorporating external information before LLM generates a response. The external information has to be vector embeddings usually stored in a vector database such as pinecore, opensearch etc... This techinique is very cost and time effective comparing with fine tuning model in which the LLM model will need to be re-trained.

There are ML models available which helps creating embeddings from text or images or both(multimodel models).

In this example, we are going to expore a typical workflow of RAG.

- Step 1: Read document for embedding
- Step 2: Create embedding using amazon embedding model - amazon.titan-embed-text-v2:0
- Step 3: Create a dataframe to store the embeddings (mock vector database)
- Step 4: Create prompt for the LLM along with the context emmbedding
- Step 5: Generate content using the Meta Llama model - ``

## Step 1: Read documents for embedding
We will use Pypdf to read a PDF document. For re-usability purpose, lets create a function to read a pdf file.

In [1]:
# Use pypdf to read PDF document as string
# Install pypdf if not exists already
!pip install pypdf

Looking in indexes: https://pypi.python.org/simple


In [2]:
from pypdf import PdfReader

#Lets create a function to read PDF pages

def as_text(pdf_file_name):
    reader = PdfReader(pdf_file_name)
    print(f'Number of pages {len(reader.pages)}')
    # Lets read only the first page
    page = reader.pages[0]
    return page.extract_text()

#print(as_text('sports_news.pdf'))

## Step 2: Create embedding using amazon embedding model
We are going to use Amazon embedding model - amazon.titan-embed-text-v2:0

Request format for the model is as below,
```
{
    "inputText": string,
    "dimensions": int,
    "normalize": boolean,
    "embeddingTypes": list
}
```
Response format,
```
{
    "embedding": [float, float, ...],
    "inputTextTokenCount": int,
    "embeddingsByType": {"binary": [int,..], "float": [float,...]}
}
```
Reference documentation - https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-embed-text.html


In [8]:
# Configure AWS environment
# Below method is NOT recommended for public sharing or deployments to environments
import os

#os.environ['AWS_ACCESS_KEY_ID'] = 'your key id'
#os.environ['AWS_SECRET_ACCESS_KEY'] = 'your secret'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [4]:
# Lets use boto3 to connect to bedrock APIs to access the model
import boto3
# Use json to build the request
import json

In [6]:
# For reuseability lets create a function
def get_text_embedding(text_for_embedding):
    bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')
    req = json.dumps({'inputText':text_for_embedding})
    response = bedrock_runtime.invoke_model(body=req, modelId='amazon.titan-embed-text-v2:0')
    response_body = json.loads(response.get('body').read())
    return response_body['embedding']
#print(get_text_embedding(as_text('sports_news.pdf')))


## Step 3: Create a data frame to store the embeddings (mock vector database)¶
In production like environments, the embeddings will be stored in a vector database. For the purpose of demonstration, lets create a mock vector database using pandas dataframe. The dataframe need to have the below,

- Identifier/title
- Text
- Embedding
-------------------------------
identifier | Text | Embedding
-------------------------------
       |               |
       |               |
       |               |       
-------------------------------

Lets use the functions that we have created on steps 1 and 2 to create the mock vector database.


In [9]:
# Use pandas to create dataframe
import pandas as pd

sports_news = as_text('sports_news.pdf')
employment_news = as_text('uk-job-market-news.pdf')
data = {'identifier':['cricket-news', 'uk-employment-news'],
       'text':[sports_news,employment_news],
        'embeddings':[get_text_embedding(sports_news), get_text_embedding(employment_news)]
       }
vector_db = pd.DataFrame(data)
vector_db

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)


Number of pages 1
Number of pages 1


,identifier,text,embeddings
0,cricket-news,Legendary India batter Virat Kohli has announc...,"[-0.03001970238983631, 0.026122447103261948, 0..."
1,uk-employment-news,The UK's job market has continued to weaken wi...,"[0.023377733305096626, 0.07752429693937302, -0..."


### Cosign Similarity
The cosine similarity formula measures the similarity between two vectors by calculating the cosine of the angle between them. It's defined as the dot product of the two vectors divided by the product of their magnitudes (Euclidean norms). The formula can be represented as:
```
cosine_similarity(A, B) = (A · B) / (||A|| * ||B||)
```
**Where:**

- (A · B): is the dot product of vectors A and B.
- ||A|| and ||B|| are the Euclidean norms (magnitudes) of vectors A and B

**This formula results in a value between -1 and 1, where:**

- A value of 1 indicates that the vectors have the same direction and are perfectly similar.
- A value of 0 indicates that the vectors are perpendicular and have no similarity.
- A value of -1 indicates that the vectors have opposite directions and are perfectly dissimilar.
